In [17]:
import sys
sys.path.append("../src/")

%load_ext autoreload
%autoreload 2

import scdeepaa

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt


In [19]:
dataset_name = 'MMD-MA'
dataset_color = 'lime'
modality_names = ['Modality 1', 'Modality 2']
data_folder = '../../JAMIE/examples/data/UnionCom/MMD/'
data1 = np.loadtxt(data_folder + "s1_mapped1.txt")
data2 = np.loadtxt(data_folder + "s1_mapped2.txt")
type1 = np.loadtxt(data_folder + "s1_type1.txt")
type2 = np.loadtxt(data_folder + "s1_type2.txt")
type1 = type1.astype(int)
type2 = type2.astype(int)
type1 = np.array([f'Cell Type {i}' for i in type1])
type2 = np.array([f'Cell Type {i}' for i in type2])

# Labels
labels = [type1, type2]
features = [None, None]
feature_dict = {}

In [39]:
res = scdeepaa.fit_deepAA(
    [data1, data2],
    [np.ones(data1.shape[0]), np.ones(data2.shape[0])],
    ["G", "G"],
    hidden_dims_dec_common = [128,64],
    hidden_dims_dec_last = [512],
    hidden_dims_enc_ind = [512],
    hidden_dims_enc_common = [256],
    hidden_dims_enc_pre_Z = [128, 64],
    lr = 0.001,
    gamma_lr = 0.01,
    linearize_encoder = False,
    linearize_decoder = False,
    steps = 2000,
    narchetypes = 3,
    batch_size = 200,
    fix_Z = False)

ELBO: 2562360.18750  :   0%|          | 2/2000 [00:00<01:49, 18.29it/s]

Fitting full model!


ELBO: 2309204.37500  : 100%|██████████| 2000/2000 [01:35<00:00, 20.86it/s]
/home/salvatore.milite/miniconda3/envs/scdeepaa/lib/python3.11/site-packages/pyro/primitives.py:137: RuntimeWarning: trying to observe a value outside of inference at loss
  warnings.warn(


In [ ]:
import scanpy as sc
import anndata as ad

In [ ]:
data1_adata = ad.AnnData(data1)
data2_adata = ad.AnnData(data2)

sc.tl.pca(data1_adata, svd_solver='arpack')
sc.tl.pca(data2_adata, svd_solver='arpack')

sc.pp.neighbors(data1_adata)
sc.pp.neighbors(data2_adata)


sc.tl.umap(data1_adata)
sc.tl.umap(data2_adata)

In [ ]:
data1_adata, arc_names1 = scdeepaa.add_to_obs_adata(res, data1_adata)
data2_adata, arc_names2 = scdeepaa.add_to_obs_adata(res, data2_adata)

In [ ]:
sc.pl.umap(data1_adata, color=arc_names1)

In [ ]:
sc.pl.umap(data2_adata, color=arc_names2)

In [26]:
import mudata as md
from mudata import MuData 
import muon as mu

ModuleNotFoundError: No module named 'mudata'

In [ ]:
mdata = MuData({"data1": data1_adata, "data2": data2_adata})
mdata

In [ ]:
_ = mu.tl.mofa(mdata, n_factors = 3)


In [ ]:
def add_MOFA_to_obs_adata(muon, adata):
    col_names = [ "factor" + str(i + 1) for i in np.arange(muon.obsm['X_mofa'].shape[1])]
    for i in range(len(col_names)):
        adata.obs[col_names[i]] = muon.obsm['X_mofa'][:,i]
    return adata, col_names

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]

plt.rcParams["figure.autolayout"] = True



data2_adata, factor_names2 = add_MOFA_to_obs_adata(mdata, data2_adata)
sc.pl.umap(data2_adata, color=factor_names2, return_fig=True) 
plt.suptitle(f"MOFA+ on simulated data",fontsize = "xx-large", y = 1.05 , x = 0.45)
plt.show()
sc.pl.umap(data2_adata, color=arc_names2, return_fig=True) 
plt.suptitle(f"Deep AA on simulated data",fontsize = "xx-large", y = 1.05 , x = 0.45)
plt.show()
